In [1]:
import time
import re
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
# driver.implicitly_wait(10)

In [4]:
driver.get('https://hsreplay.net/meta/#tab=tierlist')
time.sleep(10)

# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "myDynamicElement"))
#     )
# finally:
#     driver.quit()

In [7]:
driver.execute_script('return document.readyState;')

'complete'

In [8]:
innerHTML = driver.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML, 'html.parser')
all_decks = soup.find_all("li", class_="archetype-list-item")
links = list(map(lambda tag: tag.find("a").get('href'), all_decks))

deck_name_extract = re.compile('[^\/]+$')
base_url = 'https://hsreplay.net'
winrates = {}
df_wr = pd.DataFrame()

for link in links[:3]:
    deck = deck_name_extract.search(link).group()
    url = base_url + link + '#tab=matchups'
    print(f'{deck}:{url}')
    driver.get(url)
    time.sleep(5)
    innerHTML = driver.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML, 'html.parser')
    matchups = soup.find_all("a",class_="table-cell", attrs={"aria-describedby": re.compile("column0")})
    winrates[deck] = list(map(lambda matchup: (float(matchup.contents[0].rstrip('%')), deck_name_extract.search(matchup.get('href')).group()), matchups))
    
    zlst = list(zip(*winrates[deck]))
    s_wr = pd.Series(zlst[0], index=zlst[1])
    
    df_wr[deck] = s_wr

In [16]:
driver.quit()

df_wr = df_wr.T
df_wr = df_wr[list(df_wr.index.values)]
df_wr.to_csv(f"hsreplay_winrates_{datetime.today().strftime('%Y%m%d')}.csv")

In [ ]:
# File approach
# for i, line in enumerate(lines):
#     if line.startswith('Versus'):
#         matchups.append((' '.join(lines[i-1].split()),' '.join(lines[i].split()[1:]), float(' '.join(lines[i+1].split()[1:]).rstrip('%'))))